# Climpyrical Demo

This notebook shows some basic ways to manipulate CanRCM4 models using `climpyrical` levraging `xarray` and `numpy`.

In [ ]:
# install climpyrical
!pip install -e ../../

In [ ]:
# performs data IO
from climpyrical.data import read_data
# masking functions
from climpyrical.mask import *
# various grid operations on CanRCM4 model
from climpyrical.gridding import *

from pkg_resources import resource_filename

## Data I/O Operations

In [ ]:
# path to CanRCM4 Model (example included in climpyrical)
model_path = resource_filename('climpyrical', 'data/model_inputs/snw_rain_CanRCM4-LE_ens35_1951-2016_max_rl50_load_ensmean.nc')

In [ ]:
read_data?

In [ ]:
# read the data
model = read_data(model_path)
model

In [ ]:
# Access dataset attributes using xarray
model.rlon.size, model.rlat.size

In [ ]:
# view dataset attributes
model.keys()

In [ ]:
# detect the design value variable
(dv ,) = model.data_vars
dv

# Resizing the CanRCM4 Model

In [ ]:
# re-grid the dataset into any new resolution (non-interpolated)
N = 3
model_regridded = regrid_ensemble(model, dv, N, copy=True)
model_regridded.rlon.size, model_regridded.rlat.size

In [ ]:
N = 5
model_regridded = regrid_ensemble(model, dv, N, copy=True)
model_regridded.rlon.size, model_regridded.rlat.size

In [ ]:
# display a CanRCM initialization if 3D
import matplotlib.pyplot as plt
# choose first initialization
plt.imshow(model[dv], origin='lower')

# Performing Basic Statistics

In [ ]:
# perform ensemble statistics
import numpy as np
model_median_rlat = np.nanmedian(model[dv].values, axis=1)
model_median_rlon = np.nanmedian(model[dv].values, axis=0)

# plot slices
plt.plot(model.rlon, model_median_rlon, label="lon slice")
plt.plot(model.rlat, model_median_rlat, label="lat slice")
plt.legend()
plt.ylabel(f"{dv}")
plt.xlabel("West to East or South to North")

# Masking

In [ ]:
# load mask from file
dvmask = 'sftlf'

MASK_PATH = resource_filename('climpyrical', 'data/masks/land_mask_CanRCM4_sftlf.nc')
mask = read_data(MASK_PATH)

# convert to boolean mask
mask = mask[dvmask] > 0.

# display mask
plt.imshow(mask, origin='lower')

In [ ]:
# combine with NaN values found in CanRCM4 model
final_mask = np.logical_and(mask, ~np.isnan(model[dv].values))
plt.imshow(final_mask, origin='lower')

In [ ]:
# apply mask and create a masked model
model[dv].values[~final_mask] = np.nan
plt.imshow(model[dv].values, origin='lower')
plt.show()

# Coordinate manipulations

In [ ]:
# create paired arrays of coordinate points for every grid cell in model
x, y = flatten_coords(model.rlon.values, model.rlat.values)
x.size, y.size

In [ ]:
# get coordinate pairs for only the land values
rlon, rlat = np.meshgrid(model.rlon, model.rlat)
plt.scatter(rlon[final_mask], rlat[final_mask], marker='x', s=5)
plt.show()

In [ ]:
# coordinate matching
import pandas as pd
# load station data using pandas


DF_PATH = resource_filename('climpyrical', 'data/station_inputs/sl50_rl50_for_maps.csv')

df = pd.read_csv(DF_PATH, index_col=None)
# specify name of design value in csv file
station_dv = 'SL50 (kPa)'

# transform regular coordinates into rotated pole
rlon_st, rlat_st = transform_coords(df['lon'].values, df['lat'].values)

transform_coords?

In [ ]:
# find nearest grid cell locations in model to station coordinates
ix, iy = find_element_wise_nearest_pos(model.rlon.values, model.rlat.values, rlon_st, rlat_st)

plt.scatter(model[dv].values[iy, ix], df[station_dv])
plt.xlabel('Model Values at Observation Locations')
plt.ylabel('Observed Values')
plt.show()